In [ ]:
import math
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
df = pd.read_csv('/kaggle/input/apple-inc-aapl-historical-data/AAPL.csv')
df.head()

In [ ]:
plt.figure(figsize=(16, 8))
plt.title('Close Price History', fontsize=20)
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Cloe Price USD ($)', fontsize=18)
plt.show()

In [ ]:
# Close variable
data = df.filter(['Close'])
dataset = data.values
train_data_len = math.ceil(len(dataset) * .8)

train_data_len

In [ ]:
# Scaling data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
# Trainind Dataset
train_data = scaled_data[0:train_data_len, :]

# Split Data
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i <= 60:
        print(x_train)
        print(y_train)
        print()
        

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, 
               input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))

model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
# Testing Data set
test_data = scaled_data[train_data_len - 60: , :]

x_test = []
y_test = dataset[train_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0],
                             x_test.shape[1], 1))

In [ ]:
# Predicted price
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)


In [ ]:
# Root mean squared error
rmse = np.sqrt(np.mean(predictions - y_test) ** 2)
rmse

In [ ]:
# Plot data
train = data[:train_data_len]
valid = data[train_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16, 8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')

plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

<hr>